In [0]:
from pyspark.sql.functions import regexp_extract

tableList = spark.sql("SHOW TABLES IN gov_br_combustivel.bronze_layer")
tableList = tableList.withColumn("ano", regexp_extract("tableName", r"delta_(\d{4})_\d", 1))

In [0]:
tableList_before_2020 = tableList.filter("ano <= 2020")
display(tableList_before_2020)

In [0]:
from pyspark.sql.functions import col, regexp_replace

if tableList_before_2020.count() > 0:
    for row in tableList_before_2020.collect():
        table_name = row['tableName']
        df = spark.table(f"gov_br_combustivel.bronze_layer.{table_name}")
        df = df.withColumn("ValorCompra", regexp_replace(col("ValorCompra"), ",", "."))
        df.write.mode("overwrite").saveAsTable(f"gov_br_combustivel.bronze_layer.{table_name}")